In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import functools as ft
import numpy as np
import pandas as pd
from time import sleep
import re
from pprint import pprint
import time

In [2]:
df = pd.read_pickle(r'/home/roadcom/Downloads/naver_movie_20220221.pickle')

In [5]:
df[df.year=='2021']

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url,keywords
11951,007 노 타임 투 다이,액션,2021,09.29,6.77,"4,461",가장 강력한 운명의 적과 마주하게된 제임스 본드의 마지막 미션이 시작된다,다니엘 크레이그|라미 말렉|라샤나 린치|레아 세이두,벤 위쇼|아나 디 아르마스|나오미 해리스|랄프 파인즈|제프리 라이트|로리 키니어|빌...,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20211001_121/1...,"[(미션, 1.021356783919598), (마지막, 1.021356783919..."
11952,117편의 러브레터,멜로/로맨스|드라마,2021,03.18,8.77,13,1945년 홀로코스트에서 살아남은 헝가리 청년 '미클로시'는 \r 해방 후 폐질환으...,밀란 쉬러프|에모크 피티|안나 아즈카라테|길라 알마고,피터 쉐러,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20210302_206/1...,"[(미클로, 2.3020219011727097), (편지, 2.21327739348..."
11953,120시간: 로스트 인 아웃백,드라마|공포|스릴러,2021,05.20,4.96,25,호주로 여행을 떠나온 미국인 커플 리사와 웨이드.\r 비행기 안에서 리사가 웨이드의...,,,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20210514_75/16...,"[(호주, 1.9551399081906586), (언덕, 1.601699857829..."
11954,1984 최동원,다큐멘터리,2021,11.11,8.91,185,"무쇠팔, 부산의 심장, 최고의 투수, 등번호 11번, 불꽃 투혼, 금테 안경\r 우...",최동원,,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20211108_166/1...,"[(시리즈, 3.127373981222538), (한국, 2.537254997360..."
11955,"2021 스웨그 에이지 : 외쳐, 조선!",공연실황|뮤지컬,2021,05.13,8.20,20,삶의 고단함과 역경을 시조 속에 담아 훌훌 털어버렸던 백성들은 역모 사건으로 시조 ...,양희준|김수하,임현수|이경수|장재웅|정선기|정아영|김은총|심수영|김승용,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20210518_136/1...,"[(시조, 2.9419538879215943), (조선, 1.701969895356..."
...,...,...,...,...,...,...,...,...,...,...,...,...
12637,후겟츠 웨슬리,코미디|멜로/로맨스,2021,11.18,8.34,32,허구한 날 티격태격하는 결혼 5년차 부부 올리브와 클레이는 고민 끝에 이혼을 결심하...,알리시아 실버스톤|라이언 콴튼|랜들 바틴코프,미셸 스위니 에이브람스|에이미 J. 칼|수지 브랙|대니 로디스|페트릭 젤린스키,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20211103_72/16...,"[(웨슬리, 2.85425482474132), (반려, 2.3412846594536..."
12638,훈즈: 제국의 탄생,전쟁|액션,2021,11.18,5.86,14,기원전 3세기 중앙아시아의 스키티아. 서진한 흉노족은 투르크족을 약탈하며 세력을 키...,,,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20211115_298/1...,"[(약탈, 1.692276633838779), (부족, 1.6620164959194..."
12639,휴가,드라마,2021,10.21,8.87,76,"해고 5년차, 천막농성 1882일째\r 재복은 노조가 정리해고무효소송에서\r 최종 ...",이봉하|김아석|신운섭,김정연|이승주 |서광택|황정용|이승원|박재형,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20211006_91/16...,"[(휴가, 1.907397515134253), (해고, 1.5032138103016..."
12640,흐린 하늘에 웃다,드라마|액션,2021,06.16,4.80,5,메이지 유신 이후 일본.\r 300년마다 흐린 하늘에 나타나는 파괴의 신 '오로치'...,후쿠시 소우타|나카야마 유마|후루카와 유우키,키리야마 렌|다이토 슌스케|코세키 유타|이치카와 토모히로|카지 마사키,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20210513_280/1...,"[(파괴, 1.1217203899795918), (텐, 1.1217203899795..."


In [2]:
today_mm, today_dd = list(map(int,time.strftime('%m.%d').split('.')))
today_mm, today_dd


(2, 19)

In [3]:
year_max, year_min = 2021,2020
year_pages = []
print('Range : %d ~ %d'%(year_min,year_max))
for info in BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie_open.nhn'),'html.parser').find('table').select('td'):
    if year_min <= int(info.text[:4]) <= year_max:
        year, pages = info.text[:4], str(np.ceil(float(info.text[7:-1])/20).astype(int))
        print('Year '+year+' - '+pages+' pages')
        year_pages.append((year,pages))
year_pages

Range : 2020 ~ 2021
Year 2021 - 38 pages
Year 2020 - 37 pages


[('2021', '38'), ('2020', '37')]

In [4]:
csvdata,csvtotal = [], 0
for year, pages in year_pages:
    for page in range(1,int(pages)+1):
        try:
            soup = BeautifulSoup(urlopen('https://movie.naver.com/movie/sdb/browsing/bmovie.nhn?open='+year+'&page='+str(page)),'html.parser')
            collected, total = 0, 0
            for link in soup.find('ul','directory_list').select('li > a'):
                err_msg = ''
                try:
                    date = re.findall('\d{2}.\d{2}',link.text)

                    if 'href' in link.attrs and 'class' not in link.attrs:
                        basic_page = 'https://movie.naver.com'+link.attrs['href']
                        data = BeautifulSoup(urlopen(basic_page),'html.parser')
                        total += 1

                        # title
                        title = data.find('h3','h_movie').select('a')[0].text

                        msg = 'error at date'
                        # date
                        date = data.find('dl','info_spec').find('dd').find_all('span')[-1].find_all('a')[-1].text[1:]
                        
                        msg = 'error at rating & voting'
                        # rating, vote_count
                        rating = str(ft.reduce(lambda x,y:x+y,[val.text for val in data.find('div','score score_left').select('div > a > em')[:4]]))
                        vote_count = data.find('div','score score_left').find('div','ly_count').em.text
                            
                        msg = 'error at genre'
                        # genre
                        genre = '|'.join(list(map(lambda x: x.text, filter(lambda x: 'genre' in x.attrs['href'], data.find('dl','info_spec').select('dd > p > span > a')))))
                        #genre = genre.replace('멜로/로맨스','로맨스')
                        
                        msg = 'error at plot'
                        #plot
                        plot = data.find_all('p','con_tx')[0].text
                        
                        msg = 'error at actors'
                        # actors
                        detail_page = basic_page.replace('basic','detail')
                        detail_soup = BeautifulSoup(urlopen(detail_page))

                        main_act,supp_act = '',''
                        main_num = sum([1 if line.text == '주연' else 0 for line in detail_soup.find_all('em','p_part')])
                        if main_num > 0:
                            for i,actor in enumerate(detail_soup.find('ul','lst_people').find_all('a','k_name')):
                                if i<main_num:
                                    main_act += actor.text+'|'
                                else:
                                    supp_act += actor.text+'|'
                            main_act,supp_act = main_act[:-1],supp_act[:-1]

                        msg = 'error at img'
                        #img_url
                        img_url = data.find('div','poster').find('img').attrs['src']

                        # data check => save
                        if rating != '0.00' and genre != '' and len(date) > 0:
                            '''
                            영화제목, 장르, 연도, 날짜, 평점, 투표수, 주연, 조연, url, 이미지링크
                            '''
                            csvdata.append([title, genre, year, date, rating, vote_count,
                                           plot, main_act, supp_act, basic_page,img_url])
                            collected += 1
                            csvtotal += 1
                        
                except Exception as e:
                    pass
            #print('year {0} page {1}/{2} - crawling {3}/{4} completed - {5}'.format(year,page,pages,collected,total,csvtotal))
            #sleep(3)
        except Exception as e:
            print('year {0} page {1}/{2} - crawling {3}/{4} failed - {5}'.format(year,page,pages,collected,total,csvtotal))
        if page % 10 == 0:
            print('Check per 10 pages - Recently appended 10 data')
            pprint(csvdata[-5:])

/opt/miniconda3/envs/ai/lib/python3.8/site-packages/bs4/__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file /opt/miniconda3/envs/ai/lib/python3.8/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


Check per 10 pages - Recently appended 10 data
[['레드 아워',
  '액션|스릴러',
  '2021',
  '09.23',
  '2.00',
  '3',
  '아동보호국에서 근무하는 카일은 아프가니스탄 파병지에서 동료를 잃었던 끔찍한 기억으로 힘든 나날을 보낸다.\r'
  '\xa0직장 동료와 맥시코 출신 미아 매니와 함께 계부이자 하원의원인 샘이 운영하는 넬슨 매장에 잠시 들렀던 카일은 매장 직원들 및 '
  '다른 고객들과 함께 자살폭탄 테러범의 인질이 된다.\r'
  '\xa0카일은 다른 인질들을 구하기 위해 테러범과 목숨을 건 싸움 끝에 테러범들을 제압하고 인질들과 함께 폭파 직전에 매장에서 '
  '탈출해야만 하는데...',
  '타이레스 깁슨|존 말코비치',
  '',
  'https://movie.naver.com/movie/bi/mi/basic.naver?code=209085',
  'https://movie-phinf.pstatic.net/20210923_101/1632377012095mnKGS_JPEG/movie_image.jpg?type=m77_110_2'],
 ['레미니센스',
  'SF|미스터리|멜로/로맨스',
  '2021',
  '08.25',
  '6.99',
  '441',
  '해수면의 상승으로 도시의 절반이 바다에 잠긴 가까운 미래.\r'
  '\xa0사람들의 머릿속을 엿보는 탐정 닉은 고객들이 잃어버린 기억에 다가가게 도와주며 위험하지만 매혹적인 세계인 과거 속을 항해한다.\r'
  '\xa0단조롭던 닉의 인생은 잃어버린 귀걸이를 찾으려는 새로운 고객 메이의 등장으로 영원히 바뀌게 된다.\r'
  '\xa0닉은 메이와 운명처럼 사랑에 빠지지만, 어느 날 메이는 흔적도 없이 사라져 버린다.\r'
  '\xa0메이의 실종에 대한 진실을 찾고자 분투하던 닉은 숨겨진 잔혹한 음모를 밝혀내고, 결국 이 질문에 대해 대답을 해야만 하는데….\r'
  '\xa0“사랑하는 이를 곁에 두기 위해, 당신

Check per 10 pages - Recently appended 10 data
[['렛 힘 고',
  '범죄|드라마|스릴러',
  '2020',
  '11.05',
  '7.03',
  '224',
  '은퇴한 보안관 조지(케빈 코스트너)와 부인 마거릿(다이안 레인)이 죽은 아들의 아내 로나가 재혼한 위보이 일가로 인해 위험에 빠진 '
  '손자를 구하고자 고군분투하는 이야기',
  '케빈 코스트너|다이안 레인',
  '카일리 카터|레슬리 맨빌|제프리 도노반|윌 브리테인|부부 스튜어트',
  'https://movie.naver.com/movie/bi/mi/basic.naver?code=194324',
  'https://movie-phinf.pstatic.net/20201006_235/1601947127968Giqbe_JPEG/movie_image.jpg?type=m77_110_2'],
 ['로그',
  '액션',
  '2020',
  '11.25',
  '4.22',
  '73',
  '대장 ‘샘’(메간 폭스)이 이끄는 용병팀 ‘로그’는\r'
  '\xa0무장 단체 ‘알샤바브’에게 납치된\r'
  '\xa0주지사의 딸을 구출하기 위해\r'
  '\xa0남아프리카 공화국으로 파견된다.\r'
  '\xa0\r'
  '\xa0가까스로 타깃 구출에 성공하지만\r'
  '\xa0잔혹하고 무자비한 ‘알샤바브’의 추격은 계속되고,\r'
  '\xa0치열한 전투 속 생사의 갈림길에 서게 된 ‘로그’ 팀 앞에\r'
  '\xa0치명적인 미지의 존재가 등장한다.\r'
  '\xa0\r'
  '\xa0최정예 특수부대, 마지막 임무를 완수하라!',
  '메간 폭스',
  '필립 윈체스터|제시카 서튼|리-앤 리벤버그|브랜든 오렛|애덤 디콘|그렉 크릭',
  'https://movie.naver.com/movie/bi/mi/basic.naver?code=195596',
  'https://movie-phinf.pstatic.net/20201105_54/1604567615992

In [5]:
len(csvdata)

1369

In [6]:
csvdata[0]

['007 노 타임 투 다이',
 '액션',
 '2021',
 '09.29',
 '6.77',
 '4,461',
 '가장 강력한 운명의 적과 마주하게된 제임스 본드의 마지막 미션이 시작된다',
 '다니엘 크레이그|라미 말렉|라샤나 린치|레아 세이두',
 '벤 위쇼|아나 디 아르마스|나오미 해리스|랄프 파인즈|제프리 라이트|로리 키니어|빌리 매그너슨|다비드 덴칙|달리 벤살라',
 'https://movie.naver.com/movie/bi/mi/basic.naver?code=99702',
 'https://movie-phinf.pstatic.net/20211001_121/16330536363187liBB_JPEG/movie_image.jpg?type=m77_110_2']

In [7]:
df = pd.read_pickle('./dataset/movies04293.pickle')

In [15]:
columns = ['title','genre','year','date','rating','vote_count','plot','main_act','supp_act','page_url','img_url']

df1 = pd.DataFrame(csvdata, columns=columns)

In [16]:
df_merge= pd.concat([df[columns],df1])

In [20]:
df_merge.to_pickle('./dataset/naver_movie_20220219.o')

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act,page_url,img_url
0,1917,드라마|전쟁,2020,2.19,8.88,5017,제1차 세계대전이 한창인 1917년. 독일군에 의해 모든 통신망이 파괴된 상황 속에...,조지 맥케이|딘-찰스 채프먼,콜린 퍼스|베네딕트 컴버배치|마크 스트롱|앤드류 스캇|리차드 매든|에드리언 스카보로...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200212_161/1...
1,21 브릿지: 테러 셧다운,범죄|드라마|스릴러,2020,1.01,7.84,579,"뉴욕 맨해튼 중심에서 벌어진 경찰 연쇄 살해 사건, 범인을 잡기 위해 베테랑 경찰 ...",채드윅 보스만|J.K. 시몬스|시에나 밀러|테일러 키취,스테판 제임스|빅토리아 카타게나|키스 데이빗|샤이나 라이언|알렉산더 시디그|모로코 ...,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20191227_205/1...
2,n번째 이별중,코미디|멜로/로맨스,2020,4.01,6.48,177,여자친구 ‘데비’에게 대차게 차여 충격받은 물리학 천재 ‘스틸먼’. 사랑하는 그녀의...,에이사 버터필드|소피 터너|스카일러 거손도,오브리 레이놀즈|윌 펠츠|질리안 조이|케이든 J. 그레고리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200409_56/15...
3,가슴 큰 태희,멜로/로맨스,2020,2.27,8.67,3,곧 결혼을 앞둔 예비 신혼 부부 차욱과 민주. 차욱은 직장 후배인 용우네 부부에게 ...,NaN,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200211_88/15...
4,개인교습,멜로/로맨스,2020,3.02,10.0,1,부잣집 프랑스 아가씨 마고에게 첫눈에 반한 발리의 가난한 어부 에카는 그녀에게 피아...,하리 산티카|도르카스 카핀,NaN,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://movie-phinf.pstatic.net/20200226_62/15...
...,...,...,...,...,...,...,...,...,...,...,...
1364,황무지 5월의 고해,드라마,2020,10.28,6.95,42,"1980년 5월, 모두가 깊은 상처로 남은 그날, 행불자가 된 남자 ‘칸트’와 죄책...",조선묵|서갑숙|김영석|김윤태,,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20201021_82/16...
1365,후계자들,액션,2020,04.23,7.57,14,인도 유명 마피아 가문의 수장이 여러 압박을 이기다 못해 끝내 자살을 택한다. 해...,브렌든 프레이저|로닛 로이,프렘 초프라|아브헤이 데올|니라이 카비|샤롯 푸트렐|애니샤 빅토르|알리 하지,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20200416_297/1...
1366,후쿠오카,드라마,2020,08.27,6.63,218,책방 단골 ‘소담’ 때문에 불쑥 후쿠오카에 도착한 ‘제문’은 그녀와 함께 작은 술...,권해효|윤제문|박소담,야마모토 유키|오세현|조서|이가우에 케이코|미야케 소이치|사카타 히로키,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20200805_260/1...
1367,히트맨,코미디|액션,2020,01.22,6.77,"12,663",웹툰 작가가 되고 싶어 국정원을 탈출한 비밀 프로젝트 방패연 출신 전설의 암살요원...,권상우|정준호|황우슬혜|이이경|이지원,허성태|조운|허동원|이준혁|이중옥,https://movie.naver.com/movie/bi/mi/basic.nave...,https://movie-phinf.pstatic.net/20200122_181/1...
